In [4]:
import numpy as np
import tensorflow as tf
import time
import os
import math
import pandas as pd
from tqdm.notebook import tqdm
import scipy.sparse as sp
from tffm import TFFMClassifier
from tffm import TFFMRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error 
from sklearn.metrics import roc_auc_score, accuracy_score

dir_ = '../data/'
folder = 'sp_matrix_tag_one'
file_name = 'normalized_to_rating_filter_track_5_user_50.pkl'

In [6]:
df = pd.read_pickle(os.path.join(dir_, folder, file_name))
train = pd.read_pickle(os.path.join(dir_, folder, 'train_' + file_name))
train_ = train[['uid', 'tid', 'rating', 'count']]

In [7]:
train_x = sp.load_npz(os.path.join(dir_, folder, 'train_x_' + file_name + '.npz'))
train_y = np.loadtxt(os.path.join(dir_, folder, 'train_y_' + file_name[:-3] + 'csv'), delimiter=',')
test_x_sp = sp.load_npz(os.path.join(dir_, folder, 'test_x_' + file_name + '.npz'))
test_y_sp = np.loadtxt(os.path.join(dir_, folder, 'test_y_' + file_name[:-3] + 'csv'), delimiter=',')

In [8]:
unique_users = len(df['uid'].unique())
unique_tracks = len(df['tid'].unique())
l = unique_users + unique_tracks + 19
tid_start = unique_users
tag_start = unique_users + unique_tracks
print ('Tid start position : ' + str(tid_start))
print ('Tag start position : ' + str(tag_start))
print ('Length of vector : ' + str(l))
print('----------------------------------------')
print ('Number of unique users : ' + str(unique_users))
print ('Number of unique tracks : ' + str(unique_tracks))

Tid start position : 218
Tag start position : 39355
Length of vector : 39374
----------------------------------------
Number of unique users : 218
Number of unique tracks : 39137


In [9]:
track_ids = []
for i in range(unique_tracks):
    track_ids.append(i)

all_tracks = pd.DataFrame()
all_tracks['tid'] = track_ids
all_tracks['count'] = 0

In [10]:
x_test = []
for i in tqdm(range(unique_users)):
    user = train_[train_['uid']==i]
    top_n = all_tracks.set_index('tid').add(user.set_index('tid'), fill_value=0).reset_index()
    top_n = top_n[top_n['count']==0]
    top_n['uid'] = i
    top_n = top_n[['uid', 'tid']]
    top_n = top_n.values.tolist()
    x_test.extend(top_n)
    

In [11]:
i = 0
sp_rows = []
sp_cols = []
for row in tqdm(x_test):
    sp_rows.append(i)
    sp_cols.append(row[0])
    
    sp_rows.append(i)
    sp_cols.append(tid_start + row[1])
    
    t = df[df['tid']==row[1]].reset_index()
    tags = t['tags'].iloc[0]
    # only one tag 
    sp_rows.append(i)
    sp_cols.append(tag_start + tags)
    
#     for tag in tags:
#         sp_rows.append(i)
#         sp_cols.append(tag_start + tag)
    i += 1

In [12]:
w = np.ones(len(sp_rows))
test_x = sp.csr_matrix((w, (sp_rows, sp_cols)), shape=(len(x_test), l), dtype=float)

In [38]:
print(test_x[0,:])

  (0, 0)	1.0
  (0, 220)	1.0
  (0, 39355)	1.0


In [14]:
sp.save_npz(os.path.join(dir_, folder, 'topN_test_x_' + file_name), test_x)

In [9]:
test_x = sp.load_npz(os.path.join(dir_, folder, 'topN_test_x_' + file_name + '.npz'))

In [27]:
order = 3
model = TFFMRegressor(
    order=order, 
    rank=10, 
    optimizer=tf.train.AdamOptimizer(learning_rate=0.0001), 
    n_epochs=10, 
    batch_size=16384,
    init_std=0.00001,
    reg=0.0001,
    input_type='sparse'
)
model.fit(train_x, train_y, show_progress=True)
predictions = model.predict(test_x)

/home/han/anaconda3/envs/fm/lib/python3.6/site-packages/numpy/core/_asarray.py:136: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order, subok=True)
100%|██████████| 10/10 [00:07<00:00,  1.32epoch/s]


In [28]:
len(predictions), predictions

(8169189,
 array([0.1048109 , 0.10758879, 0.09725256, ..., 0.09666096, 0.09734733,
        0.09340532], dtype=float32))

In [29]:
df2 = pd.DataFrame(x_test, columns=['uid', 'tid'])
df2.insert(2, 'rating', predictions, False) 
df2[:10]

,uid,tid,rating
0,0,2,0.104811
1,0,5,0.107589
2,0,7,0.097253
3,0,28,0.096753
4,0,37,0.106504
5,0,39,0.106587
6,0,52,0.091209
7,0,53,0.109771
8,0,57,0.096947
9,0,64,0.103708


In [30]:
# df2.to_pickle(os.path.join(dir_, folder, 'topN_pred_' + file_name[:-3] + 'pkl'))

In [31]:
predictions = model.predict(test_x_sp)
print('RMSE: {}'.format(math.sqrt(mean_squared_error(test_y_sp, predictions))))
print('MAE: {}'.format(mean_absolute_error(test_y_sp, predictions)))

RMSE: 2.588479022771308
MAE: 1.9248533108091392


In [32]:
predictions = model.predict(train_x)
print('RMSE: {}'.format(math.sqrt(mean_squared_error(train_y, predictions))))
print('MAE: {}'.format(mean_absolute_error(train_y, predictions)))

RMSE: 2.5861427547633995
MAE: 1.9230153750982533


In [33]:
predictions = df2

In [34]:
test = pd.read_pickle(os.path.join(dir_, folder, 'test_' + file_name[:-3] + 'pkl'))
test.sort_values(by=['uid','tid'])[:10]

,uid,tid,count,tags,rating
7,0,2,4,0,3.00
12,0,5,2,4,1.50
14,0,7,5,1,3.50
44,0,28,2,4,1.50
53,0,37,1,12,0.25
55,0,39,3,1,2.50
74,0,52,1,1,0.25
75,0,53,1,1,0.25
80,0,57,4,1,3.00
88,0,64,1,2,0.25


In [35]:
num_user = len(predictions['uid'].unique())
num_user

218

In [36]:
# Personal Recommendation
n=20
top_n_lists = []

for i in tqdm(range(num_user)):
    prediction = predictions[predictions['uid'] == i]
    prediction = prediction.sort_values(by=['rating'],  ascending=False)
    prediction = prediction[:n]
#     print(prediction)
    top_n_list = []
    for _, row in prediction.iterrows():
        top_n_list.append(row[1])
#     print(top_n_list)
    top_n_lists.append(top_n_list)

evaluation = []
satisfication = 0
for i in tqdm(range(num_user)):
    top_n_list = top_n_lists[i]
    precision = 0
    for j in top_n_list:
        p = test[test['uid'] == i]
        p = p[p['tid']==j]
        if len(p) > 0:
            precision += 1
    satisfication += precision / n
    evaluation.append(precision)
#     print(precision, satisfication)
print(satisfication/num_user)


0.12339449541284414
